In [2]:
pip install matplotlib

     ---------------------------------------- 7.6/7.6 MB 18.8 MB/s eta 0:00:00
     ---------------------------------------- 56.2/56.2 KB ? eta 0:00:00
     ---------------------------------------- 436.0/436.0 KB ? eta 0:00:00
     ---------------------------------------- 103.1/103.1 KB ? eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 27.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Nayeon\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [4]:
pip install seaborn

     -------------------------------------- 294.6/294.6 KB 6.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Nayeon\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from konlpy.tag import Komoran
import matplotlib.pyplot as plt

# 데이터 하나로 합치기
- 실제 문장이 들어있는 칼럼만 사용
- 통합된 데이터는 csv로 저장해두기

In [6]:
movie = pd.read_csv("영화리뷰.csv")
purpose = pd.read_csv("용도별목적대화데이터.csv")
topic = pd.read_csv("주제별일상대화데이터.csv")
common_sense = pd.read_csv("일반상식.csv")
add = pd.read_csv("추가데이터.csv")

movie.dropna(inplace=True)
purpose.dropna(inplace=True)
topic.dropna(inplace=True)
common_sense.dropna(inplace=True)
add.dropna(inplace=True)

In [7]:
print(f"movie shape => {movie.shape}\n\
purpose shape => {purpose.shape}\n\
topic shape => {topic.shape}\n\
common_sense shape => {common_sense.shape}\n\
add => {add.shape}")

movie shape => (199992, 3)
purpose shape => (480494, 1)
topic shape => (1445760, 1)
common_sense shape => (68538, 3)
add => (9216, 2)


In [8]:
all_data = list(movie['document']) + list(purpose['text']) + list(topic['text']) + list(common_sense['query']) \
+ list(common_sense['answer']) + list(add['query'])

In [9]:
len(all_data)

2272538

In [12]:
df = pd.DataFrame({'text': all_data})
df.to_csv("./통합데이터.csv", index=False)

# 의도 분류 데이터 생성
- 0 -> 장소
- 1 -> 시간
- 2 -> 사람
- 3 -> 내용
- 4 -> 기타

In [24]:
what = []
where = []
when = []
who = []
etc = []

In [35]:
for i in all_data:
    if ('위치' or '어디' or '장소' or '주소') in i: where.append(i)
    elif ('시작' or '마감' or '언제' or '기간' or '시간' or '며칠' or '몇일' or '몇 일' or '요일' or '몇 시' or '이번 주' or '주말' or '다음 주' or '지난 주') in i: when.append(i)
    elif ('누구' or '누가' or '사람') in i: who.append(i)
    elif ('뭐') in i: what.append(i)
    else: etc.append(i)

In [36]:
len(who)

21460

In [37]:
where_label = [0] * len(where)
when_label = [1] * len(when)
who_label = [2] * len(who)
what_label = [3] * len(what)
etc_label = [4] * len(etc)


In [38]:
len(who_label)

21460

In [39]:
train_df = pd.DataFrame({'text': where + when + who + what + etc,
                         'label' : where_label + when_label + who_label + what_label + etc_label})

train_df.head()

,text,label
0,2014년 한국사회 안에서 예술가라는 포지션이 어디쯤에 위치하고 있는가에 대해서 깊...,0
1,짐캐리를 지금의 위치로 올려준 94년도 3작품중 하나,0
2,"홍콩에서 가장 유능한 감독 유위강, 당시 최고의 배우 정이건, 만화계의 절대적인 위...",0
3,80년대 국산 애니메이션 중 보석같은 작품중의 하나 입니다. 이런 애니메이션이 계속...,0
4,평점주기전에 시즌오브더위치 보고올것..시즌오브더위치보다 평점이 낮다니 말도안되,0


In [40]:
train_df.tail()

,text,label
4545071,흡연충이 언제 슈퍼영웅이 되었냐?,4
4545072,페이스북에 잘난부모둔게죄냐는 글올린 당당함은 언제그랫었냐는듯 전혀다른사람같은태도군,4
4545073,언제환기시킬수 있는지 모르겠다 거의 미세먼지라서 그런지,4
4545074,스스로나가면 스스로나가면 연봉 안줘도되고 짜르면 남은 기간 연봉 다줘야하고 그게 팩트다,4
4545075,언제까지 인터넷여론이 진짜 여론인줄알거야? ㅋㅋ,4


In [41]:
train_df.to_csv("train_data.csv", index=False)